In [1]:
import matplotlib.pyplot as plt
import torch
import spatialSSL

file_path = "../example_files/img_1199670929.h5ad"

# Create the dataloader
dataset_constructor = spatialSSL.Dataloader.EgoNetDatasetConstructor(file_path=file_path, image_col="section",
                                                                     label_col="class_label", include_label=False,
                                                                     radius=20, node_level=3)

# Load the data
dataset_constructor.load_data()

# Construct the graph
dataset = dataset_constructor.construct_graph(True)
#dataset = torch.load("../dataset/dataset2.pt")

total_cells = len(dataset_constructor.adata)
print(len(dataset), "of", total_cells)

Processing 1 images:   0%|          | 0/1 [00:00<?, ?it/s]

Processing 26230 nodes:   0%|          | 0/26230 [00:00<?, ?it/s]

24778 of 26230


In [2]:
# Construct the graph
#dataset = dataset_constructor.construct_graph()
#dataset = torch.load("../dataset/dataset2.pt")

total_cells = len(dataset_constructor.adata)
print(len(dataset), "of", total_cells)

24778 of 26230


In [3]:
# test new Training.py
train_loader, val_loader, test_loader = spatialSSL.Utils.split_dataset(dataset=dataset, split_percent=(0.8, 0.1, 0.1), batch_size = 64)


pretrain_weights = torch.load("/Users/leopoldendres/Documents/Bioinformatik/MasterStudium/spatial_atlas_ssl/models/model_2_20_0.9_GCN.pt", map_location=torch.device('cpu'))








pretrain_weights
#torch.save(dataset, "../dataset/dataset2.pt")

OrderedDict([('preconv1.bias',
              tensor([ 0.1466,  0.8431,  0.1395, -0.4153,  0.5844, -0.3269,  0.2011,  0.2596,
                      -0.1122, -0.0133,  0.7679, -0.1675,  0.2850,  0.5696,  0.1298, -0.1109,
                       0.1179,  1.0377, -0.3357, -0.0749, -0.3219, -0.2919, -0.3573, -0.0899,
                      -0.7089,  0.4092, -0.3035,  0.2246,  0.5494, -0.1565, -0.4105, -0.5618,
                      -0.3672,  0.2295,  0.3898, -0.0584, -0.0399, -0.1789, -0.3600,  0.3952,
                      -0.5660, -0.6186, -0.3196, -0.0661,  0.1466, -0.0098, -0.2885,  0.3426,
                       0.1503, -0.2307, -0.4500, -0.2208, -0.3602,  0.6797,  0.5027, -0.0995,
                      -0.0468,  0.2296,  0.3180, -0.4975, -0.2046, -0.1110,  0.0871, -0.3174,
                      -0.5075,  0.0569,  0.4397, -0.1375, -0.6812,  0.2742,  0.1161,  0.2259,
                       0.1535, -0.2713, -0.0582,  0.0098,  0.5491,  0.0740,  0.9834, -0.7831,
                       0.1721

In [24]:
pretrain_weights#['preconv2.bias'].shape#[0].shape

OrderedDict([('preconv1.bias',
              tensor([ 0.1466,  0.8431,  0.1395, -0.4153,  0.5844, -0.3269,  0.2011,  0.2596,
                      -0.1122, -0.0133,  0.7679, -0.1675,  0.2850,  0.5696,  0.1298, -0.1109,
                       0.1179,  1.0377, -0.3357, -0.0749, -0.3219, -0.2919, -0.3573, -0.0899,
                      -0.7089,  0.4092, -0.3035,  0.2246,  0.5494, -0.1565, -0.4105, -0.5618,
                      -0.3672,  0.2295,  0.3898, -0.0584, -0.0399, -0.1789, -0.3600,  0.3952,
                      -0.5660, -0.6186, -0.3196, -0.0661,  0.1466, -0.0098, -0.2885,  0.3426,
                       0.1503, -0.2307, -0.4500, -0.2208, -0.3602,  0.6797,  0.5027, -0.0995,
                      -0.0468,  0.2296,  0.3180, -0.4975, -0.2046, -0.1110,  0.0871, -0.3174,
                      -0.5075,  0.0569,  0.4397, -0.1375, -0.6812,  0.2742,  0.1161,  0.2259,
                       0.1535, -0.2713, -0.0582,  0.0098,  0.5491,  0.0740,  0.9834, -0.7831,
                       0.1721

In [31]:
for item in list(pretrain_weights.items()):
    print(item[0], item[1].shape)

list(pretrain_weights.items())[0][1].shape[0]

preconv1.bias torch.Size([128])
preconv1.lin.weight torch.Size([128, 550])
preconv2.bias torch.Size([550])
preconv2.lin.weight torch.Size([550, 128])


128

In [20]:
model = spatialSSL.TrainModels.GCN_classifier(in_channels=550, hidden_channels=list(pretrain_weights.items())[0][1].shape[0], num_classes=len(dataset_constructor.adata.obs.class_label.unique()))

In [22]:
model.preconv1.lin.weight = torch.nn.Parameter(pretrain_weights['preconv1.lin.weight'])
model.preconv1.lin.weight# = torch.nn.Parameter(pretrain_weights['preconv1.bias'])

Parameter containing:
tensor([[-0.1370, -0.0103, -0.3340,  ..., -0.0289,  0.4123, -0.1240],
        [ 0.0541,  0.0309,  0.0179,  ..., -0.0051, -0.1280,  0.0473],
        [-0.2776,  0.2851,  0.1078,  ..., -0.0223, -0.0758, -0.0754],
        ...,
        [-0.1147, -0.1052,  0.2259,  ..., -0.0302, -0.0394,  0.2127],
        [-0.3062,  0.2560, -0.5042,  ..., -0.0095, -0.1452,  0.2553],
        [-0.0730, -0.2864, -0.1323,  ...,  0.0127, -0.0916,  0.2303]],
       requires_grad=True)

In [36]:
model = spatialSSL.TrainModels.GCN_classifier(in_channels=550, hidden_channels=list(pretrain_weights.items())[0][1].shape[0], num_classes=len(dataset_constructor.adata.obs.class_label.unique()))

# costum layer initialization
model.preconv1.weight = torch.nn.Parameter(pretrain_weights['preconv1.lin.weight'])
model.preconv1.bias = torch.nn.Parameter(pretrain_weights['preconv1.bias'])


#model.load_state_dict(pretrain_weights, strict=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


training_summary = spatialSSL.Training.train(model, train_loader, val_loader, optimizer, criterion, 100, 10, model_path = "../downstream_models/output_test.pt", gene_expression=dataset_constructor.adata)#, masking_ratio=masking_mode)



model



Training model:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100, train loss: 2.2243, train r2: 0.6340,  val loss: 2.1900, val r2: 0.6629, Time: 20.24s
Epoch 2/100, train loss: 2.1761, train r2: 0.6775,  val loss: 2.1779, val r2: 0.6729, Time: 20.51s
Epoch 3/100, train loss: 2.1730, train r2: 0.6800,  val loss: 2.1760, val r2: 0.6748, Time: 21.79s
Epoch 4/100, train loss: 2.1674, train r2: 0.6852,  val loss: 2.1781, val r2: 0.6732, Time: 23.25s
Epoch 5/100, train loss: 2.1703, train r2: 0.6815,  val loss: 2.1869, val r2: 0.6644, Time: 27.26s
Epoch 6/100, train loss: 2.1694, train r2: 0.6835,  val loss: 2.1723, val r2: 0.6793, Time: 36.51s
Epoch 7/100, train loss: 2.1664, train r2: 0.6872,  val loss: 2.1800, val r2: 0.6710, Time: 28.60s
Epoch 8/100, train loss: 2.1682, train r2: 0.6844,  val loss: 2.1704, val r2: 0.6804, Time: 24.24s
Epoch 9/100, train loss: 2.1651, train r2: 0.6866,  val loss: 2.1679, val r2: 0.6831, Time: 25.22s
Epoch 10/100, train loss: 2.1647, train r2: 0.6884,  val loss: 2.1634, val r2: 0.6881, Time: 26.98s
Epoch 11/


KeyboardInterrupt



In [9]:
dataset_constructor.adata.obs

,fov,volume,center_x,center_y,min_x,min_y,max_x,max_y,barcodeCount,corrected_x,...,supertype_label,nt_type_combo_label,CCF_landmark,CCF_bin_x,CCF_bin_y,CCF_bin_z,structures_id,CCF_acronym,CCF_analysis,CCF_broad
1015221640100570419,True,521.489213,1313.267618,2804.310553,1307.762779,2799.531567,1318.574285,2808.685538,True,2291.554254,...,MY Calcb Chol,Chol,TBD,0.0,0.0,0.0,0,NA,NA,NA
1015221640101030179,True,434.644809,2337.988188,4469.224479,2332.236809,4464.972032,2343.273111,4472.625751,True,4164.954813,...,SPVI-SPVC Mafa Glut_2,Glut,SPVC,0.0,0.0,0.0,0,NA,NA,NA
1015221640101040298,True,1301.728682,2753.101886,4534.981912,2744.089138,4527.718707,2762.242116,4541.635168,True,4335.910947,...,SPVI-SPVC Mafa Glut_2,Glut,SPVC,0.0,0.0,0.0,0,NA,NA,NA
1015221640101040300,True,949.511632,2525.507108,4535.021918,2518.233714,4528.029029,2531.782917,4542.876204,True,4277.043727,...,SPVI-SPVC Mafa Glut_2,Glut,SPVC,0.0,0.0,0.0,0,NA,NA,NA
1015221640100930227,True,1832.142257,2816.284756,4036.023695,2806.999792,4024.808109,2825.768915,4046.358749,True,3870.307249,...,SPVI-SPVC Mafa Glut_2,Glut,SPVC,0.0,0.0,0.0,0,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015221640100240734,True,739.178047,1348.739592,1433.912120,1342.893678,1427.961523,1355.041469,1439.032137,True,977.031838,...,Astro-NT NN_2,NA,TBD,0.0,0.0,0.0,0,NA,NA,NA
1015221640100710728,True,454.965108,2434.388524,3390.977631,2430.263258,3385.330893,2439.302937,3396.104476,True,3148.398579,...,Astro-NT NN_2,NA,TBD,0.0,0.0,0.0,0,NA,NA,NA
1015221640100340405,True,1470.242392,963.977757,1906.023814,955.829571,1896.588464,972.537121,1914.268653,True,1333.473025,...,Astro-NT NN_2,NA,TBD,0.0,0.0,0.0,0,NA,NA,NA
1015221640100350582,True,976.467347,1345.458493,1860.768403,1338.098138,1854.138736,1353.193144,1866.964757,True,1388.494134,...,Astro-NT NN_2,NA,TBD,0.0,0.0,0.0,0,NA,NA,NA


In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, training=True):
    model.train(training)
    losses = []
    accuracies = []
    for batch in data_loader:
        optimizer.zero_grad()
        x = batch.x.to(device)
        edge_index = batch.edge_index.to(device)
        y = batch.y.to(device)
        out = model(x, edge_index)
        loss = F.cross_entropy(out, y)
        losses.append(loss.item())
        if training:
            loss.backward()
            optimizer.step()
        acc = (out.argmax(dim=1) == y).sum().item() / len(y)
        accuracies.append(acc)
    return mean(losses), mean(accuracies)

In [4]:
net = GCN(in_channels=550, hidden_channels=550, nu=550)

#weigths = torch.load("../models/best_model.pt")

# load model state
#net.load_state_dict(torch.load("../models/best_model.pt"),  strict=False)
#weigths

TypeError: __init__() got an unexpected keyword argument 'out_channels'

In [111]:
weigths

OrderedDict([('conv1.bias',
              tensor([ 1.8485e-01,  5.2149e-02,  2.5553e-01,  1.6856e-01,  2.3748e-01,
                      -3.8776e-02,  1.3727e-01,  2.3419e-01,  1.7705e-01,  2.0290e-01,
                       1.7215e-01,  9.0370e-02,  2.4223e-01,  8.7808e-02,  2.3943e-02,
                       2.1018e-01,  1.6180e-01,  2.3478e-01,  2.4502e-01,  2.8955e-01,
                       1.5057e-01,  1.2783e-01,  2.5978e-01,  1.2592e-01,  2.7632e-01,
                       2.6478e-01, -1.7233e-02,  1.4562e-01,  1.9541e-01,  8.0785e-03,
                       1.6502e-01,  1.5623e-01,  1.5175e-01,  1.0052e-01,  1.9142e-01,
                       5.7890e-02,  1.3251e-01,  1.6412e-01,  6.0202e-02,  1.8642e-01,
                       1.5235e-01,  1.6511e-01,  2.1999e-01,  3.1011e-01,  2.2046e-02,
                       1.9942e-01,  2.5617e-01,  1.4763e-01,  2.0891e-01,  9.1363e-02,
                       1.9775e-01,  1.7344e-01,  1.4490e-01,  1.8761e-01,  1.9027e-01,
               

In [109]:
net.state_dict()

OrderedDict([('conv1.bias',
              tensor([ 1.8485e-01,  5.2149e-02,  2.5553e-01,  1.6856e-01,  2.3748e-01,
                      -3.8776e-02,  1.3727e-01,  2.3419e-01,  1.7705e-01,  2.0290e-01,
                       1.7215e-01,  9.0370e-02,  2.4223e-01,  8.7808e-02,  2.3943e-02,
                       2.1018e-01,  1.6180e-01,  2.3478e-01,  2.4502e-01,  2.8955e-01,
                       1.5057e-01,  1.2783e-01,  2.5978e-01,  1.2592e-01,  2.7632e-01,
                       2.6478e-01, -1.7233e-02,  1.4562e-01,  1.9541e-01,  8.0785e-03,
                       1.6502e-01,  1.5623e-01,  1.5175e-01,  1.0052e-01,  1.9142e-01,
                       5.7890e-02,  1.3251e-01,  1.6412e-01,  6.0202e-02,  1.8642e-01,
                       1.5235e-01,  1.6511e-01,  2.1999e-01,  3.1011e-01,  2.2046e-02,
                       1.9942e-01,  2.5617e-01,  1.4763e-01,  2.0891e-01,  9.1363e-02,
                       1.9775e-01,  1.7344e-01,  1.4490e-01,  1.8761e-01,  1.9027e-01,
               

In [95]:
len(dataset_constructor.adata.obs.class_label.unique())#.cat.codes.values

['MY GABA', 'MY Glut', 'Astro-Epen', 'Oligo', 'P GABA', ..., 'CNU-HYa GABA', 'MB Glut', 'CNU-HYa Glut', 'TH Glut', 'MB Dopa']
Length: 20
Categories (20, object): ['Astro-Epen', 'CB GABA', 'CB Glut', 'CNU-HYa GABA', ..., 'P GABA', 'P Glut', 'TH Glut', 'Vascular']

In [96]:
for batch in dataset:
    print(batch.x)
    break

tensor([    0,  3600, 17561])


In [98]:
for batch in train_loader:
    print(batch.x)
    break

tensor([ 1302,  6712,  6727,  ..., 39346, 40173, 50669])


In [5]:
from tqdm.auto import tqdm
from torch import optim
from statistics import mean
from torcheval.metrics import MulticlassAccuracy


train_loader, val_loader, test_loader = spatialSSL.Utils.split_dataset(dataset=dataset, split_percent=(0.8, 0.1, 0.1), batch_size = 64)

epochs = 20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = GCN(in_channels=550, hidden_channels=550, num_classes=len(dataset_constructor.adata.obs.class_label.unique()))

#net.load_state_dict(torch.load("../models/full_model.pt",  map_location=torch.device(device)), strict=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)#, momentum=0.9)




net.to(device)
# Train the model
# training loop
for epoch in tqdm(range(epochs)):
    net.train()


    running_loss = 0.0

    train_acc = MulticlassAccuracy()
    val_acc = MulticlassAccuracy()

    for data in train_loader:
        #inputs, _ = data.
        input = torch.tensor(dataset_constructor.adata.X[data.x].toarray(), dtype=torch.double).to(device).float()

        labels = torch.tensor(dataset_constructor.adata[data.x.numpy()].obs.class_label.cat.codes.values).to(device).long()
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(input, data.edge_index)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        #accuracy = (outputs.argmax(dim=1) == labels).sum().item() / len(labels)
        train_acc.update(outputs.argmax(dim=1), labels)
        #train_accs.append(accuracy)
        running_loss += loss.item()

    running_loss = running_loss / len(train_loader)
       # Validation
    net.eval()
    val_accs = []
    with torch.no_grad():
        val_loss = 0.0
        for data in val_loader:

            input = torch.tensor(dataset_constructor.adata.X[data.x].toarray(), dtype=torch.double).to(device).float()
            labels = torch.tensor(dataset_constructor.adata[data.x.numpy()].obs.class_label.cat.codes.values).to(device).long()
            outputs = net(input, data.edge_index)
            loss = criterion(outputs, labels)

            val_acc.update(outputs.argmax(dim=1), labels)

            #accuracy = (outputs.argmax(dim=1) == labels).sum().item() / len(labels)
            #val_accs.append(accuracy)
            val_loss += loss.item()

        #val_accuracy = 100 * correct / total
        avg_val_loss = val_loss / len(val_loader)



    print(f'Epoch: {epoch+1}/{epochs}, Train Loss: {running_loss:.4f}, Val Loss: {avg_val_loss:.4f} Train Acc: {train_acc.compute():.4f}, Val Acc: {val_acc.compute():.4f}')



    #print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.3f}, accuracy: {sum(all_accuracy) / len(all_accuracy):.3f}')

  0%|          | 0/20 [00:00<?, ?it/s]


KeyboardInterrupt



In [7]:
from tqdm.auto import tqdm
from torch import optim
from statistics import mean
from torcheval.metrics import MulticlassAccuracy

epochs = 20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = GCN(in_channels=550, hidden_channels=550, num_classes=len(dataset_constructor.adata.obs.class_label.unique()))

#net.load_state_dict(torch.load("../models/full_model.pt",  map_location=torch.device(device)), strict=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)#, momentum=0.9)




net.to(device)
# Train the model
# training loop
for epoch in tqdm(range(epochs)):
    net.train()


    running_loss = 0.0

    train_acc = MulticlassAccuracy()
    val_acc = MulticlassAccuracy()

    for data in train_loader:
        #inputs, _ = data.
        input = torch.tensor(dataset_constructor.adata.X[data.x].toarray(), dtype=torch.double).to(device).float()

        labels = torch.tensor(dataset_constructor.adata[data.x.numpy()].obs.class_label.cat.codes.values).to(device).long()
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(input, data.edge_index)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        #accuracy = (outputs.argmax(dim=1) == labels).sum().item() / len(labels)
        train_acc.update(outputs.argmax(dim=1), labels)
        #train_accs.append(accuracy)
        running_loss += loss.item()

    running_loss = running_loss / len(train_loader)
       # Validation
    net.eval()
    val_accs = []
    with torch.no_grad():
        val_loss = 0.0
        for data in val_loader:

            input = torch.tensor(dataset_constructor.adata.X[data.x].toarray(), dtype=torch.double).to(device).float()
            labels = torch.tensor(dataset_constructor.adata[data.x.numpy()].obs.class_label.cat.codes.values).to(device).long()
            outputs = net(input, data.edge_index)
            loss = criterion(outputs, labels)

            val_acc.update(outputs.argmax(dim=1), labels)

            #accuracy = (outputs.argmax(dim=1) == labels).sum().item() / len(labels)
            #val_accs.append(accuracy)
            val_loss += loss.item()

        #val_accuracy = 100 * correct / total
        avg_val_loss = val_loss / len(val_loader)



    print(f'Epoch: {epoch+1}/{epochs}, Train Loss: {running_loss:.4f}, Val Loss: {avg_val_loss:.4f} Train Acc: {train_acc.compute():.4f}, Val Acc: {val_acc.compute():.4f}')



    #print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.3f}, accuracy: {sum(all_accuracy) / len(all_accuracy):.3f}')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1/20, Train Loss: 2.6104, Val Loss: 2.4669 Train Acc: 0.5257, Val Acc: 0.6286
Epoch: 2/20, Train Loss: 2.4438, Val Loss: 2.4517 Train Acc: 0.6449, Val Acc: 0.6324
Epoch: 3/20, Train Loss: 2.4395, Val Loss: 2.4484 Train Acc: 0.6452, Val Acc: 0.6336
Epoch: 4/20, Train Loss: 2.4346, Val Loss: 2.4379 Train Acc: 0.6492, Val Acc: 0.6473
Epoch: 5/20, Train Loss: 2.4257, Val Loss: 2.4348 Train Acc: 0.6580, Val Acc: 0.6492
Epoch: 6/20, Train Loss: 2.4218, Val Loss: 2.4333 Train Acc: 0.6611, Val Acc: 0.6502
Epoch: 7/20, Train Loss: 2.4218, Val Loss: 2.4320 Train Acc: 0.6603, Val Acc: 0.6511
Epoch: 8/20, Train Loss: 2.4230, Val Loss: 2.4321 Train Acc: 0.6592, Val Acc: 0.6507
Epoch: 9/20, Train Loss: 2.4216, Val Loss: 2.4312 Train Acc: 0.6603, Val Acc: 0.6512
Epoch: 10/20, Train Loss: 2.4202, Val Loss: 2.4310 Train Acc: 0.6620, Val Acc: 0.6512
Epoch: 11/20, Train Loss: 2.4149, Val Loss: 2.4311 Train Acc: 0.6674, Val Acc: 0.6518
Epoch: 12/20, Train Loss: 2.4194, Val Loss: 2.4300 Train Acc: 0


KeyboardInterrupt



In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, training=True):
    model.train(training)
    losses = []
    accuracies = []
    for batch in data_loader:
        optimizer.zero_grad()
        x = batch.x.to(device)
        edge_index = batch.edge_index.to(device)
        y = batch.y.to(device)
        out = model(x, edge_index)
        loss = F.cross_entropy(out, y)
        losses.append(loss.item())
        if training:
            loss.backward()
            optimizer.step()
        acc = (out.argmax(dim=1) == y).sum().item() / len(y)
        accuracies.append(acc)
    return mean(losses), mean(accuracies)